<a href="https://colab.research.google.com/github/FIJY/Lesson_01/blob/master/hh/%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_hh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Парсинг с сайта hh <br>
Ссылка на видео парсинга hh https://yandex.ru/video/preview/11468178118806870731

In [ ]:
# Устанавливаем библиотеки
# requests для отправки запросов
# BeautifulSoup для разбора полученных данных
# lxml для парсинга
# fake_useragent для создания заголовков
!pip install requests
!pip install bs4
!pip install lxml
!pip install fake_useragent

In [ ]:
# Импортируем библиотеки
import requests
from bs4 import BeautifulSoup
import fake_useragent
import time
import json

In [ ]:
import pandas as pd

In [ ]:
import re

In [ ]:
# Заходим на hh, выбираем регион (Россия)
# Вводим запрос в поле поиска (например python) в разделе Соискателям
# Переходим на вкладку резюме
# Переходим на вторую страницу, чтобы появился параметр page
# Копируем адрес страницы

In [ ]:
# Функция для получения ссылок по запросу
def get_links(text):
  ua = fake_useragent.UserAgent() #Создаем объект для передачи заголовка
  # Пишем запрос и передаем туда скопированный адрес страницы hh
  # В скопированном адресе находим текст нашего запроса (python) и заменяем его на параметр функции (text)

  data = requests.get(
      url=f"https://hh.ru/search/resume?text={text}&area=1&isDefaultArea=true&exp_period=all_time&logic=normal&pos=full_text&page=1",
      headers = {"user-agent":ua.random})

  # Если произошла ошибка, выходим из функции
  if data.status_code != 200:
    return

  # В переменную soup передаем содержание полученной страницы
  soup = BeautifulSoup(data.content, "lxml")
  # Найдем количество страниц
  # Внизу открытой страницы hh находим количество страниц 250; нажимаем правой кнопкой мыши и далее "Проверить"
  # В открывшемся справа окне, вверху нажимаем кнопку проверить и выбираем количество страниц (250)
  # Находим div class = "pager", и вводим его в attrs
  # Далее надо найти все "span" и взять последний, т.к. там указано 250
  # Далее у последнего span надо получить ссылку "a"
  # В ссылке надо взять "span", и в "span" взять текст
  # Полученное значение переводим в int
  try:
    page_count = int(soup.find("div", attrs={"class":"pager"}).find_all("span",recursive=False)[-1].find("a").find("span").text)
  except:
    return

  for page in range(page_count):
    try:
      # Создаем заголовок и передаем туда случайный заголовок из созданного объекта
      # В запрос передаем страницу
      data = requests.get(
        url=f"https://hh.ru/search/resume?text={text}&area=1&isDefaultArea=true&exp_period=all_time&logic=normal&pos=full_text&page={page}",
        headers = {"user-agent":ua.random}
      )

      if data.status_code != 200:
        continue
      soup = BeautifulSoup(data.content, "lxml")
      # Выбираем на hh любое резюме, нам нужна ссылка "a", у нее нужен атрибут "data-qa"; этот атрибут вставляем в soup
      # Проходим по всем ссылкам
      for a in soup.find_all("a", attrs={"data-qa":"serp-item__title"}):
        # в каждой ссылке "a" нам нужен адрес сайта в "attrs"
        # в адресе сайта нужно всё до знака "?", поэтому разбиваем ссылку по знаку "?" и берем первый элемент
        yield f"https://hh.ru{a.attrs['href'].split('?')[0]}"
    except Exception as e:
      print(f"{e}")
    time.sleep(1)

In [ ]:
# Полученные ссылки соберем в список link_resume
link_resume = [a for a in get_links("python")]

In [ ]:
len (link_resume)

4999

In [ ]:
# Запишем полученные ссылки в файл
with open ('link_resume.txt', 'w') as file:
  for resume in link_resume:
    file.write(f"{resume} ")

In [ ]:
# Переходим по любой ссылке в резюме, с указанной зарплатой
# Смотрим код и копируем class заголовка (для этого нажмем "проверить" и выделим заголовок, т.е. название позиции, например Python developer)
# Аналогично скопируем class зарплаты
# Также ищем class ключевых навыков

In [ ]:
# Функция для получения данных резюме
def get_resume(link):
  ua = fake_useragent.UserAgent() # Создадим юзер-агента
  # Делаем запрос к переданной ссылке
  data = requests.get(
      url = link,
      headers={"user-agent":ua.random}
  )
  if data.status_code != 200:
    return

  soup = BeautifulSoup(data.content, "lxml")

  # Спарсим данные о соискателе
  reperson = re.compile(r'resume-personal.*') # Регулярное выражение для поиска всех атрибутов, содержащих "resume-personal"
  try:
    persons = [person.text.replace("\xa0", " ") for person in soup.find_all(attrs={"data-qa":reperson})]
  except:
    persons = ""

  # Спарсим готовность к переезду, командировкам
  try:
    moving = soup.find(attrs={"class":"bloko-translate-guard"}).text.replace("\xa0", " ")
  except:
    moving = ""

  # Спарсим название позиции
  try:
    name = soup.find(attrs={"class":"resume-block__title-text"}).text
  except:
    name = ""

  # Спарсим зарплату и уберем лишние символы (\u2009, \xa0), которые увидели в "сыром" виде (см. код ниже)
  try:
    salary = soup.find(attrs={"class":"resume-block__salary"}).text.replace("\u2009","").replace("\xa0", " ")
  except:
    salary = ""

  # Спарсим специализации и график работы; добавив пробелы перед заглавными буквами и затем удалив лишние пробелы (в начале)
  try:
    specialization_schedule = re.sub(r'([А-ЯA-Z])', r' \1', soup.find(attrs={"class":"resume-block-container"}).text).strip()
  except:
    specialization_schedule = ""

  # Спарсим опыт работы
  try:
    experience = soup.find(attrs={"data-qa":"resume-block-experience"}).text.replace("\xa0", " ")
  except:
    experience = ""

  # Находим атрибут, где находятся все ключевые навыки (data-qa = "skills-table); далее находим class для каждого отдельного навыка, оставляем class с text
  try:
    tags = [tag.text for tag in soup.find(attrs={"data-qa":"skills-table"}).find_all(attrs={"class":"bloko-tag__section_text"})]
  except:
    tags = []


  resume = {
      "persons":persons,
      "moving":moving,
      "name":name,
      "salary":salary,
      "specialization_schedule":specialization_schedule,
      "experience":experience,
      "tags":tags
  }
  return resume

In [ ]:
# Здесь в зарплате видны лишние символы (\u2009, \xa0), скопируем их и вставим в созданную выше функцию, чтобы их убрать
for a in link_resume[:10]:
  print(get_resume(a))
  time.sleep(1)

{'name': 'Программист Python', 'salary': ''}
{'name': 'Python разработчик', 'salary': ''}
{'name': 'Python developer', 'salary': '270\u2009000\xa0₽ на\xa0руки'}
{'name': 'Python разработчик', 'salary': ''}
{'name': 'Программист Python', 'salary': ''}
{'name': 'Python-разработчик', 'salary': ''}
{'name': 'Python-разработчик', 'salary': '80\u2009000\xa0₽ на\xa0руки'}
{'name': 'Python-разработчик/ Python developer', 'salary': ''}
{'name': 'Python Backend Developer', 'salary': ''}
{'name': 'Python developer', 'salary': ''}


In [ ]:
# Вот, что получилось, когда убрали лишние символы в зарплате
for a in link_resume[:10]:
  print(get_resume(a))
  time.sleep(1)

{'name': 'Программист Python', 'salary': ''}
{'name': 'Python разработчик', 'salary': ''}
{'name': 'Python developer', 'salary': '270000 ₽ на руки'}
{'name': 'Python разработчик', 'salary': ''}
{'name': 'Программист Python', 'salary': ''}
{'name': 'Python-разработчик', 'salary': ''}
{'name': 'Python-разработчик', 'salary': '80000 ₽ на руки'}
{'name': 'Python-разработчик/ Python developer', 'salary': ''}
{'name': 'Python Backend Developer', 'salary': ''}
{'name': 'Python developer', 'salary': ''}


In [ ]:
# Добавили навыки
for a in link_resume[:10]:
  print(get_resume(a))
  time.sleep(1)

{'name': 'Программист Python', 'salary': '', 'tags': ['JavaScript', 'Python', 'Linux', 'HTML5', 'SQL', 'CSS3', 'Git', 'SQLite', 'Django Framework', 'jQuery', 'PostgreSQL', 'Debian']}
{'name': 'Python разработчик', 'salary': '', 'tags': ['Python', 'Django REST Framework', 'Django Framework', 'PostgreSQL', 'MySQL', 'SQLite', 'Unit Testing', 'Git', 'GitHub', 'API', 'Telegram API', 'Linux', 'Ubuntu Server', 'Nginx', 'Docker', 'Gunicorn', 'Postfix', 'Postman', 'PyCharm', 'Visual Studio Code', 'Navicat']}
{'name': 'Python developer', 'salary': '270000 ₽ на руки', 'tags': ['Ответственность', 'Исполнительность', 'JavaScript', 'Node.js', 'Python', 'react.js', 'Git', 'CSS', 'Django Framework', 'SQL', 'vue.js', 'Flask', 'PostgreSQL']}
{'name': 'Python разработчик', 'salary': '', 'tags': [' SQL', 'Docker', 'Flask', 'MySQL', 'PostgreSQL', 'FastAPI', 'ООП', 'REST API', 'Django Rest Framework', 'Django Framework', 'Паттерны проектирования', 'Redis', 'Celery', ' Python', ' Git', 'RabbitMQ', 'SOLID']}


In [ ]:
for a in link_resume[:10]:
  print(get_resume(a))
  time.sleep(1)

{'persons': ['Мужчина', '47 лет', '18 мая 1976', 'Ногинск'], 'moving': 'Ногинск, готов к переезду (Москва, Россия), не готов к командировкам', 'name': 'Программист Python', 'salary': '', 'specialization_schedule': 'Специализации: Программист, разработчик Занятость: полная занятость, частичная занятость, проектная работа, стажировка График работы: полный день, гибкий график, удаленная работа', 'experience': 'Опыт работы 8 лет 6 месяцев Ноябрь 2015 — по настоящее время8 лет 6 месяцевИндивидуальное предпринимательство / частная практика / фрилансПрограммист PythonПрактическое применение знаний и навыков программирования в области web, а также изучение новых технологий в сфере программирования (всё, что связано с python в основном).', 'tags': ['JavaScript', 'Python', 'Linux', 'HTML5', 'SQL', 'CSS3', 'Git', 'SQLite', 'Django Framework', 'jQuery', 'PostgreSQL', 'Debian']}
{'persons': ['Мужчина', '37 лет', '12 марта 1987', 'Санкт-Петербург'], 'moving': 'Санкт-Петербург, готов к переезду (Моск

In [ ]:
# Сохраняем результат в файл .json
data = []
for a in link_resume[:10]:
  data.append(get_resume(a))
  time.sleep(1)
  # Сохраняем в файл
  with open("data.json", "w", encoding="utf-8") as f:
    json.dump(data,f,indent=4,ensure_ascii=False)

In [ ]:
# Извлекаем данные из файла .json с помощью pandas
data_df = pd.read_json("data.json")

In [ ]:
data_df

,persons,moving,name,salary,specialization_schedule,experience,tags
0,"[Мужчина, 47 лет, 18 мая 1976, Ногинск]","Ногинск, готов к переезду (Москва, Россия), не...",Программист Python,,"Специализации: Программист, разработчик Занято...",Опыт работы 8 лет 6 месяцев Ноябрь 2015 — по н...,"[JavaScript, Python, Linux, HTML5, SQL, CSS3, ..."
1,"[Мужчина, 37 лет, 12 марта 1987, Санкт-Петербург]","Санкт-Петербург, готов к переезду (Москва), го...",Python разработчик,,"Специализации: Программист, разработчик Занято...",Опыт работы 12 лет 8 месяцев Декабрь 2022 — по...,"[Python, Django REST Framework, Django Framewo..."
2,"[Мужчина, 25 лет, 19 октября 1998, Москва]","Москва, готов к переезду, готов к командировкам",Python developer,270000 ₽ на руки,270 000 ₽ на руки,Опыт работы 8 лет 2 месяца Июнь 2020 — по наст...,"[Ответственность, Исполнительность, JavaScript..."
3,"[Мужчина, 24 года, 4 июня 1999, Москва]","Москва, не готов к переезду, готов к редким ко...",Python разработчик,,"Специализации: Программист, разработчик Занято...",Опыт работы 2 года 6 месяцев Ноябрь 2023 — по ...,"[ SQL, Docker, Flask, MySQL, PostgreSQL, FastA..."
4,"[Мужчина, 23 года, 13 мая 2000, Москва]","Москва, готов к переезду, готов к командировкам",Программист Python,,"Специализации: Программист, разработчик Занято...",Опыт работы 2 года 2 месяца Май 2023 — по наст...,"[C/C++, Pandas, MS SQL Server, Python, Flask,..."
5,"[Мужчина, 21 год, 13 сентября 2002, Москва, м....","Москва, м. ВДНХ, не готов к переезду, не готов...",Python-разработчик,,"Специализации: Программист, разработчик Занято...",Опыт работы 2 года 4 месяца Ноябрь 2021 — Февр...,"[Python, ООП, Git, PostgreSQL, SQL, Django Fra..."
6,"[Мужчина, 38 лет, 17 марта 1986, Казань]","Казань, хочу переехать (Москва), готов к коман...",Python-разработчик,80000 ₽ на руки,80 000 ₽ на руки,Опыт работы 12 лет 4 месяца Сентябрь 2013 — по...,[]
7,"[Мужчина, 23 года, 23 июня 2000, Москва, м. Сх...","Москва, м. Сходненская, не готов к переезду, г...",Python-разработчик/ Python developer,,"Специализации: Программист, разработчик Занято...",Опыт работы 4 года 2 месяца Декабрь 2020 — по ...,"[Исполнительность, Поиск информации в интернет..."
8,"[Мужчина, 31 год, 28 февраля 1993, Москва, м. ...","Москва, м. Строгино, готов к переезду, готов к...",Python Backend Developer,,"Специализации: Программист, разработчик Занято...",Опыт работы 2 года 3 месяца Февраль 2022 — по ...,"[Python, Linux, PostgreSQL, Docker, Git, Djang..."
9,"[Мужчина, 24 года, 10 декабря 1999, Новосибирс...","Новосибирск, м. Заельцовская, готов к переезду...",Python developer,,"Специализации: Программист, разработчик Занято...",Опыт работы 2 года 4 месяца Март 2023 — по нас...,"[Git, Python, ООП, Django Framework, HTML5, CS..."
